INSTALL PACKAGES

In [1]:
# !pip install --upgrade accelerate peft bitsandbytes transformers pdfplumber datasets==2.16.1 trl==0.7.9 --target=/kaggle/working/packages
%pip install --upgrade accelerate peft transformers pdfplumber datasets==2.16.1 trl
%pip install bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui


  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
  Using cached peft-0.9.0-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached pdfplumber-0.11.0-py3-none-any.whl.metadata (39 kB)
  Using cached trl-0.7.11-py3-none-any.whl.metadata (10 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
Using cached peft-0.9.0-py3-none-any.whl (190 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached pdfplumber-0.11.0-py3-none-any.whl (56 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached trl-0.7.11-py3-none-any.whl (155 kB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20221105
    Uninstalling pdfminer.six-20221105:
      Successfully uninstalled pdfminer.six-20221105
  Rolling back uninstall of pdfminer.six
  Moving to c:\users\user\appdata\roaming\pyt

ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'c:\\Python310\\Scripts\\dumppdf.py'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/bitsandbytes-windows-webui
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


**To resolve some issues while installing packages**

In [2]:
# import pkg_resources

# def resolve_conflicts():
#     """
#     Resolves dependency conflicts by creating a new environment and installing compatible versions.
#     """

#     # Create a new virtual environment (replace "myvenv" with your desired name)
#     !virtualenv myvenv
#     !source myvenv/bin/activate

#     # Install compatible versions of conflicting packages
#     required_versions = {
#         "cupy-cuda11x": ">=12.0.0",
#         "dill": "<0.3.2,>=0.3.1.1",
#         "numpy": "<1.25.0,>=1.14.3",
#         "pyarrow": "<10.0.0,>=3.0.0",
#         "jupyter-server": "~=1.16",
#         "jupyterlab": "~=3.4",
#         "urllib3": "<2.1,>=1.25.4",  # Ensure python_version >= "3.10"
#         "pandas": "<1.6.0dev0,>=1.3",
#         "protobuf": "<5,>=4.21",
#         "dask": "==2023.7.1",
#         "distributed": "==2023.7.1",
#         "fsspec": "==2023.6.0",
#         "urllib3": "<2.0",
#         "google-api-core[grpc]": "<2.0.0dev,>=1.22.2",
#         "packaging": "<22.0dev,>=14.3",
#         "grpcio": "<2.0dev,>=1.51.3",
#         "jupyter-lsp": ">=2.0.0",
#         "google-cloud-storage": "<3,>=2.2.1",
#         "shapely": ">=2.0.1",
#         "numpy": "<1.25,>=1.21",
#         "cryptography": "<42,>=38.0.0",
#         "numpy": "<1.22.2,>=1.15.0",
#         "scipy": "<1.8.0,>=1.7.3",
#         "fsspec": "==2023.12.2",
#         "typing-extensions": "<4.6.0,>=3.6.6",
#         "platformdirs": "<4,>=2.4",
#         "numpy": "<1.24,>=1.16.0",
#         "pandas": "!=1.4.0,<2.1,>1.1",
#         "scipy": "<1.12,>=1.4.1",
#     }

#     for package, version_spec in required_versions.items():
#         !pip install "{package} {version_spec}"

#     # Verify that conflicts are resolved
#     working_set = pkg_resources.WorkingSet()
#     if not working_set.require(required_versions):
#         raise RuntimeError("Dependency conflicts could not be resolved.")

#     print("Dependency conflicts resolved!")

# if __name__ == "__main__":
#     resolve_conflicts()


**Importing necessary packages**

In [3]:
import os
import torch
import transformers
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Model: https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v0.3?text=My+name+is+Clara+and+I+am**

In [4]:
# Model from Hugging Face hub
base_model = "PY007/TinyLlama-1.1B-Chat-v0.3"

# Fine-tuned model
new_model = "TinyLlama-chat-medaid-model"

**Reading the PDF and storing it as a text file**

In [5]:
# READING THE NEW CONVERSATION DATASET WITH ALL THE TEXT FILES (doctor-patient-conversation-large)

# Directory containing your text files
text_files_directory = r"G:\LLM-Model-MedAid-Thesis\Model-3\Dataset"

# List to store individual conversation texts
conversation_texts = []

# Loop through each text file in the directory
for filename in os.listdir(text_files_directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(text_files_directory, filename)
        with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
            # Use errors="ignore" to skip characters that cannot be decoded
            conversation_texts.append(file.read())

# Save the combined text to a file
output_path = r"G:\LLM-Model-MedAid-Thesis\Model-3\combined_conversations.txt"
with open(output_path, "w", encoding="utf-8") as output_file:
    for text in conversation_texts:
        output_file.write(text + "\n\n")

dataset = load_dataset("text", data_files=output_path, split="train")


Generating train split: 53861 examples [00:00, 1110414.60 examples/s]


In [6]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 53861
})


**This code has a configuration for quantization using the *BitsAndBytesConfig class* from the *trl library*. Quantization is a technique used in deep learning to reduce the memory and computational requirements of neural networks.**

In [7]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [8]:

from torch.utils.cpp_extension import CUDA_HOME
print(CUDA_HOME) # by default it is set to /usr/local/cuda/

C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.4


**This code is using the AutoModelForCausalLM class from the transformers library to instantiate a pre-trained causal language model with specific configurations.**

In [9]:
import torch
print("Cuda ", torch.cuda.is_available())

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    # device_map={"": 0}
    device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
    trust_remote_code=True, # prevents running custom model files on your machine
)

model.config.use_cache = False
model.config.pretraining_tp = 1


Cuda  True
bin c:\Python310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


**This code is using the AutoTokenizer class from the transformers library to instantiate a tokenizer for a pre-trained language model.**

In [10]:

# Create the tokenizer on the chosen device
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prepare Model for Training

In [11]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

**This code is defining a configuration for the LoRA (Local Reparameterization with Attention) model using the LoraConfig class, which appears to be part of the peft library.**

In [12]:
# Create a LoraConfig instance
peft_params = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, peft_params)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 720,896 || all params: 1,100,781,568 || trainable%: 0.0654894686608706


**Preparing Training Dataset**

In [13]:
# # create tokenize function
# def tokenize_function(examples):
#     # extract text
#     text = examples["example"]

#     #tokenize and truncate text
#     tokenizer.truncation_side = "left"
#     tokenized_inputs = tokenizer(
#         text,
#         return_tensors="np",
#         truncation=True,
#         max_length=512
#     )

#     return tokenized_inputs

# # tokenize training and validation datasets
# tokenized_data = dataset.map(tokenize_function, batched=True)

In [14]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

**This code is defining a set of training parameters using the TrainingArguments class, which is often used in the transformers library for configuring training settings.**

In [15]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# Specify the output directory and other training parameters
training_params = TrainingArguments(
    output_dir="./results",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_epochs,
    gradient_accumulation_steps=4,
    # save_steps=25,
    logging_strategy="epoch",
    save_strategy="epoch",
    # logging_steps=25,  
    weight_decay=0.01,
    # load_best_model_at_end=True,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",
    # bf16=False,
    # max_grad_norm=0.3,
    # max_steps=-1,
    # warmup_ratio=0.03,
    # group_by_length=True,
    # lr_scheduler_type="constant",
    # report_to="tensorboard"
)

**This code is creating an instance of the SFTTrainer class, presumably from the trl library, to facilitate the training of a model using the specified configuration.**

**SFTTrainer instance is configured with the** 
* model, 
* training dataset, 
* Peft configuration, 
* tokenizer,
* training arguments. 

**The specific behavior and training process are determined by the SFTTrainer implementation in the trl library, and the configured parameters influence aspects such as optimization, learning rate, and model architecture during training.**

In [16]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
    data_collator=data_collator
)

C:\Users\User\AppData\Roaming\Python\Python310\site-packages\trl\trainer\sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map:   0%|          | 0/53861 [00:00<?, ? examples/s]

Map: 100%|██████████| 53861/53861 [00:00<00:00, 68700.12 examples/s]


In [17]:
# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

  0%|          | 0/33660 [00:00<?, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Python310\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
 10%|█         | 3366/33660 [56:06<8:40:38,  1.03s/it] Checkpoint destination directory ./results\checkpoint-3366 already exists and is non-empty.Saving will proceed but saved results may be invalid.
c:\Python310\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils

{'loss': 3.233, 'learning_rate': 0.00018002852219383209, 'epoch': 1.0}


 20%|██        | 6733/33660 [1:52:12<7:16:51,  1.03it/s] c:\Python310\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 2.9897, 'learning_rate': 0.0001600332758928041, 'epoch': 2.0}


 30%|███       | 10099/33660 [2:48:21<7:21:09,  1.12s/it]c:\Python310\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

 90%|█████████ | 30298/33660 [8:23:06<54:58,  1.02it/s]  c:\Python310\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 2.704, 'learning_rate': 2.0072493909323193e-05, 'epoch': 9.0}


100%|██████████| 33660/33660 [9:18:24<00:00,  1.04it/s]  

{'loss': 2.7049, 'learning_rate': 1.010161031552677e-07, 'epoch': 10.0}
{'train_runtime': 33504.5782, 'train_samples_per_second': 16.076, 'train_steps_per_second': 1.005, 'train_loss': 2.864780009423277, 'epoch': 10.0}


100%|██████████| 33660/33660 [9:18:24<00:00,  1.00it/s]


**The code you provided is saving the trained model and its associated tokenizer to a specified directory using the save_pretrained method.**

In [18]:
model_path = "preTrained_model"

trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

# trainer.model.save_pretrained(new_model)
# trainer.tokenizer.save_pretrained(new_model)

('preTrained_model\\tokenizer_config.json',
 'preTrained_model\\special_tokens_map.json',
 'preTrained_model\\tokenizer.model',
 'preTrained_model\\added_tokens.json',
 'preTrained_model\\tokenizer.json')

In [19]:
# Import the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**This code creates a simple conversational loop that simulates a doctor assistant interaction. It uses a fine-tuned language model (assumed to be a text generation model) and a tokenizer to generate responses based on user input. The conversation is logged in a text file, and the loop continues until the user provides an exit signal.**

In [20]:
# logging.set_verbosity(logging.CRITICAL)

# Initial prompt
prompt = "Be a doctor assistant. And keep questioning one by one to extract symptoms and history of the patient. Don't give advice or ask anything else. Just extract symptoms or history by questioning one question at a time."

# Create a pipeline for text generation using the fine-tuned model and tokenizer
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# File to save the conversation
output_file_path = "G:\LLM-Model-MedAid-Thesis\Model-3\conversation_log.txt"  # Update 'your_folder' with the desired folder in your Drive

# Function to ask a question and get the user's response
def ask_question_and_log(prompt, user_response, file_path):
    # Ask the question
    result = pipe(f"<s>[Prompt] {prompt} \n [User response] {user_response} \n [/Response]")

    # Get the generated text (question)
    generated_text = result[0]['generated_text']

    # Print and save the generated text (question)
    print(generated_text)
    with open(file_path, "a", encoding="utf-8") as output_file:
        output_file.write(f"[Model] {generated_text}\n\n")

    # Simulate user answering the question
    user_response = input("Your response: ")  # User provides input

    return user_response

# Initial user response
user_response = "I have chest pain."

# Ask a question based on the user's response and log the conversation
with open(output_file_path, "a", encoding="utf-8") as output_file:
    output_file.write("\nConversation started.\n\n")

# Loop to continue the conversation
while True:
    user_response = ask_question_and_log(prompt, user_response, output_file_path)

    # Check for an exit condition (e.g., user response indicating the end of the conversation)
    if "exit" in user_response.lower():
        print("Ending the conversation.")

        # Save the conversation to a file
        with open(output_file_path, "a", encoding="utf-8") as output_file:
            output_file.write("\nConversation ended by user. ---------------- \n\n")

        break


Both `max_new_tokens` (=32) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<s>[Prompt] Be a doctor assistant. And keep questioning one by one to extract symptoms and history of the patient. Don't give advice or ask anything else. Just extract symptoms or history by questioning one question at a time. 
 [User response] I have chest pain. 
 [/Response] [/User]






























Both `max_new_tokens` (=32) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<s>[Prompt] Be a doctor assistant. And keep questioning one by one to extract symptoms and history of the patient. Don't give advice or ask anything else. Just extract symptoms or history by questioning one question at a time. 
 [User response] I have chest pain 
 [/Response] [User response] [User response] [User response] [User response] [User response] [User response] [User response] [User response]


Both `max_new_tokens` (=32) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<s>[Prompt] Be a doctor assistant. And keep questioning one by one to extract symptoms and history of the patient. Don't give advice or ask anything else. Just extract symptoms or history by questioning one question at a time. 
 [User response] I am 30 years old 
 [/Response]

[Prompt] Do you have any allergies? [/Prompt]
[User response] [/User response]

[


Both `max_new_tokens` (=32) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<s>[Prompt] Be a doctor assistant. And keep questioning one by one to extract symptoms and history of the patient. Don't give advice or ask anything else. Just extract symptoms or history by questioning one question at a time. 
 [User response] yes 
 [/Response]


































Both `max_new_tokens` (=32) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<s>[Prompt] Be a doctor assistant. And keep questioning one by one to extract symptoms and history of the patient. Don't give advice or ask anything else. Just extract symptoms or history by questioning one question at a time. 
 [User response] i have pain in my chest 
 [/Response] 
[User response] [Tone] [Phrase] [Phrase] [Phrase] [Phrase] [P
Ending the conversation.
